<a href="https://colab.research.google.com/github/BinduGGowda/Cost-comparsion/blob/main/Cosr-Comparsion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pandas openpyxl

In [11]:

from google.colab import files

print("Upload PPR cost analysis file:")
uploaded_ppr = files.upload()

print("Upload PRD cost analysis file:")
uploaded_prd = files.upload()

file_ppr = list(uploaded_ppr.keys())[0]
file_prd = list(uploaded_prd.keys())[0]


Upload PPR cost analysis file:


Saving LSEGSaaSPPR_CostAnalysis_Sep2025.xlsx to LSEGSaaSPPR_CostAnalysis_Sep2025 (3).xlsx
Upload PRD cost analysis file:


Saving LSEGSaaSPRD_CostAnalysis_Sep2025 1.xlsx to LSEGSaaSPRD_CostAnalysis_Sep2025 1 (3).xlsx


In [22]:
# Step 3: Set output file name
output_file = "Dynamic_Azure_Cost_Optimization_Report.xlsx"

# Step 4: Process and generate report
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference

# Load both cost analysis files
df_ppr = pd.read_excel(file_ppr, sheet_name="Data", engine="openpyxl")
df_prd = pd.read_excel(file_prd, sheet_name="Data", engine="openpyxl")

# Aggregate cost by ResourceType and Squad
agg_ppr = df_ppr.groupby(["ResourceType", "Squad"])["CostUSD"].sum().reset_index()
agg_prd = df_prd.groupby(["ResourceType", "Squad"])["CostUSD"].sum().reset_index()

# Merge for comparison
comparison = pd.merge(agg_ppr, agg_prd, on=["ResourceType", "Squad"], suffixes=("_PPR", "_PRD"), how="outer").fillna(0)
comparison["Cost_Diff"] = comparison["CostUSD_PRD"] - comparison["CostUSD_PPR"]

# Summary
total_ppr = df_ppr["CostUSD"].sum()
total_prd = df_prd["CostUSD"].sum()
summary = pd.DataFrame({"Environment": ["PPR", "PRD"], "TotalCostUSD": [total_ppr, total_prd]})

# Top 10 differences
top10 = comparison.sort_values(by="Cost_Diff", key=abs, ascending=False).head(10)

# Create Excel workbook
wb = Workbook()

# Detailed Comparison sheet
ws1 = wb.active
ws1.title = "Detailed Comparison"
for r in dataframe_to_rows(comparison, index=False, header=True):
    ws1.append(r)

# Summary sheet
ws2 = wb.create_sheet(title="Summary")
for r in dataframe_to_rows(summary, index=False, header=True):
    ws2.append(r)

# Top 10 Differences sheet
ws3 = wb.create_sheet(title="Top10 Differences")
for r in dataframe_to_rows(top10, index=False, header=True):
    ws3.append(r)

# Add bar chart for Summary
chart = BarChart()
chart.title = "Total Cost by Environment"
chart.x_axis.title = "Environment"
chart.y_axis.title = "Total Cost (USD)"
data = Reference(ws2, min_col=2, min_row=1, max_row=ws2.max_row)
cats = Reference(ws2, min_col=1, min_row=2, max_row=ws2.max_row)
chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)
ws2.add_chart(chart, "D2")



In [23]:

#Save workbook
wb.save(output_file)
print(f"✅ Report generated: {output_file}")

# Step 5: Download the report
files.download(output_file)



✅ Report generated: Dynamic_Azure_Cost_Optimization_Report.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>